In [4]:
from graph_reader import Graph
from utils import *
import numpy as np
import pandas as pd
from scipy.special import comb
import random
import time
import os

In [2]:
A = np.array([[1, 0, 1, 0, 2, 4], 
              [0, 1, 2, 4, 6, 12],
              [0, 0, 1, 0, 4, 12],
              [0, 0, 0, 1, 1, 3],
              [0, 0, 0, 0, 1, 6],
              [0, 0, 0, 0, 0, 1]])

def threepathsampler(G, k):
    count = [0,0,0,0,0,0]
    C_hat = [0,0,0,0,0,0]
    sample = random.choices(G.Edges, weights= G.tauList, k = k)
    for u,v in sample:
        u0 = sampleNbr(G.adj[u], v)
        v0 = sampleNbr(G.adj[v], u)
        S_l = (u0, u, v, v0)
        if u0==v0 or u0==None or v0==None:
            continue
        motif = determineMotifType(G,S_l)
        count[motif]+=1
    for i in range(1,6): # from 2-1, to 6-1
        C_hat[i] = (count[i]/k) * (G.W/A[1,i])
    N1 = sum([comb(G.getDegree(v),3) for v in G.adj.keys()])
    C_hat[0] = N1 - C_hat[2] - 2*C_hat[4] - 4*C_hat[5]
    return C_hat

def centeredSampler(G, k):
    count = [0,0,0,0,0,0]
    C_hat = [0,0,0,0,0,0]
    successful_samples = 0
    sample = random.choices(G.Edges, weights=G.lambdaList, k=k)
    for u, v in sample:
        u0 = sampleHigherDegNbr(G, G.adj[u], v)
        v0 = sampleHigherDegNbr(G, G.adj[v], u)
        S_l = (u0, u, v, v0)
        if u0==None or v0==None or (not G.isEdge(u0,v0)):
            continue
        motif = determineMotifType(G,S_l)
        count[motif] += 1
        successful_samples+=1

    for i in range(3, 6):
        C_hat[i] = (count[i]/k) * (G.L/A[3,i])
    return C_hat

def computeFreq(G, k=200000):
    C_hat1 = threepathsampler(G, k)
    C_hat2 = centeredSampler(G, k)
    print("1 algo: ", [f"{num:.2e}" for num in C_hat1])
    print("2 algo: ", [f"{num:.2e}" for num in C_hat2])
    C_hat = C_hat1[:3]+C_hat2[3:]
    return C_hat

In [39]:
folder =  os.path.join(os.getcwd(), 'data')
final_df = pd.DataFrame(columns =['Graph', 'countV', 'countE', 'W', 'L', 'W/L', '3star', '3path', 'tailed_triangle', '4cycle', 'chordal_4cycle', '4clique'])
for file in os.listdir(folder):    
    path = os.path.join(folder, file)
    
    start_time = time.time()
    B = Graph(path)
    print(f'Graph {file} reading time: {time.time()-start_time:.0f} s')
    print(f"Number of edges {len(B.Edges):.2e}", f"Number of nodes {len(B.V):.2e}", f"W is {B.W:.2e}, L is {B.L:.2e}, W/L is {B.W/B.L:.1f}")

    start_time = time.time()
    C_hat = computeFreq(B, k=200000)
    print(["{:.2e}".format(int(num)) for num in C_hat])
    print('Exe time: ', round(time.time()-start_time, 2), 's')

    record = {'Graph': file, 'countV': len(B.V), 'countE':len(B.Edges), 'W': B.W , 'L': B.L, 
              'W/L': B.W/B.L,'3star': C_hat[0], '3path': C_hat[1], 'tailed_triangle': C_hat[2],
              '4cycle': C_hat[3], 'chordal_4cycle': C_hat[4], '4clique': C_hat[5]}
    final_df = final_df.append(record, ignore_index=True)
final_df.to_csv('results', index=False)

Amazon0312.txt
Amazon0505.txt
Amazon0601.txt
as-skitter.txt
cit-Patents.txt
web-BerkStan.txt
web-Google.txt
WikiTalk.txt


In [ ]:
path = 'data/web-BerkStan.txt'

start_time = time.time()
B = Graph(path)
print(f'Graph web-BerkStan.txt reading time: {time.time()-start_time:.0f} s')
print(f"Number of edges {len(B.Edges):.2e}", f"Number of nodes {len(B.adj.keys()):.2e}", f"W is {B.W:.2e}, L is {B.L:.2e}, W/L is {B.W/B.L:.1f}")

start_time = time.time()
C_hat = computeFreq(B, k=200000)
print(["{:.2e}".format(int(num)) for num in C_hat])
print('Exe time: ', round(time.time()-start_time, 2), 's')

In [5]:
pd.set_option("precision", 2)
data = pd.read_csv('results.csv')

In [8]:
data = pd.read_csv('ca-coauthors.csv')

In [9]:
data

,Graph,ReadingTime,ExeTime,countV,countE,W,L,W/L,3star,3path,tailed_triangle,4cycle,chordal_4cycle,4clique
0,ca-coauthors-dblp.mtx,135.44,22.89,540486,15245729,381382721695,56465879655,6.75,2.70e+10,4.17e+10,6.69e+10,3.11e+07,3.49e+09,1.53e+10


In [10]:
data.iloc[:,3:7] = data.iloc[:,3:7].apply(lambda x: x.apply('{:.2e}'.format))

In [11]:
data

,Graph,ReadingTime,ExeTime,countV,countE,W,L,W/L,3star,3path,tailed_triangle,4cycle,chordal_4cycle,4clique
0,ca-coauthors-dblp.mtx,135.44,22.89,5.40e+05,1.52e+07,3.81e+11,5.65e+10,6.75,2.70e+10,4.17e+10,6.69e+10,3.11e+07,3.49e+09,1.53e+10
